In [1]:
#https://pstu.ru/sveden/education/?act=discip&key={9bdd1ebd-2005-4f54-9156-37d9e9ed145c}
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import PyPDF2
from pdf2image import convert_from_path
import pytesseract

# Функция для скачивания PDF файлов с URL
def download_pdfs_from_url(url):
    # Загружаем страницу по URL
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Не удалось загрузить страницу: {url}")
        return []

    # Парсим HTML с помощью BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Извлекаем все ссылки на PDF файлы
    pdf_links = []
    for a_tag in soup.find_all('a', href=True):
        href = a_tag['href']
        if href.lower().endswith('.pdf'):
            full_url = urljoin(url, href)  # Полный URL PDF
            pdf_links.append(full_url)
    
    return pdf_links

# Функция для скачивания и сохранения всех PDF файлов
def download_and_save_pdfs(url, output_dir="pdf_files"):
    # Создаем директорию для сохранения PDF, если она не существует
    os.makedirs(output_dir, exist_ok=True)

    # Получаем список всех PDF-ссылок на странице
    pdf_links = download_pdfs_from_url(url)
    print(f"Найдено {len(pdf_links)} PDF файлов.")

    # Скачиваем каждый PDF файл
    for pdf_url in pdf_links:
        try:
            # Загружаем PDF файл
            response = requests.get(pdf_url)
            if response.status_code == 200:
                # Получаем имя файла из URL
                pdf_name = os.path.basename(urlparse(pdf_url).path)
                pdf_path = os.path.join(output_dir, pdf_name)

                # Сохраняем PDF
                with open(pdf_path, 'wb') as f:
                    f.write(response.content)
                print(f"Скачан: {pdf_name}")

                # Попытка извлечь текст из PDF
                pdf_text = extract_text_from_pdf(pdf_path)

                # Если текст не найден, выполняем OCR
                if not pdf_text:
                    pdf_text = extract_text_with_ocr(pdf_path)

                # Сохраняем текст в .txt файл
                text_file_path = os.path.join(output_dir, f"{os.path.splitext(pdf_name)[0]}.txt")
                with open(text_file_path, 'w', encoding='utf-8') as text_file:
                    text_file.write(pdf_text)
                print(f"Текст извлечен и сохранен в: {text_file_path}")

        except Exception as e:
            print(f"Ошибка при скачивании {pdf_url}: {e}")

# Функция для извлечения текста из PDF с помощью PyPDF2
def extract_text_from_pdf(pdf_path):
    try:
        # Открываем PDF файл
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ''
            # Проходим по всем страницам и извлекаем текст
            for page_num in range(len(reader.pages)):
                page = reader.pages[page_num]
                text += page.extract_text()
            return text
    except Exception as e:
        print(f"Не удалось извлечь текст из PDF {pdf_path}: {e}")
        return ""

# Функция для извлечения текста с помощью Tesseract OCR, если PDF содержит только изображения
def extract_text_with_ocr(pdf_path):
    try:
        # Конвертируем страницы PDF в изображения
        images = convert_from_path(pdf_path)
        text = ''
        
        # Для каждой страницы используем Tesseract для распознавания текста
        for image in images:
            text += pytesseract.image_to_string(image, lang='rus')
        
        return text
    except Exception as e:
        print(f"Не удалось извлечь текст из PDF с OCR {pdf_path}: {e}")
        return ""

# Основная функция
def main():
    # Ввод URL
    url = input("Введите URL страницы для скачивания PDF: ").strip()

    # Скачиваем и сохраняем PDF файлы
    download_and_save_pdfs(url)

if __name__ == "__main__":
    main()


Найдено 79 PDF файлов.
Скачан: {53495773-0d08-4188-8bf4-948e916c12d4}.pdf
Текст извлечен и сохранен в: pdf_files\{53495773-0d08-4188-8bf4-948e916c12d4}.txt
Скачан: {28f91b30-895f-4d84-a5ff-1a0335ecd559}.pdf
Текст извлечен и сохранен в: pdf_files\{28f91b30-895f-4d84-a5ff-1a0335ecd559}.txt
Скачан: {5a352c2d-dfad-48c4-811c-b245d97cef5c}.pdf
Текст извлечен и сохранен в: pdf_files\{5a352c2d-dfad-48c4-811c-b245d97cef5c}.txt
Скачан: {9c4620ae-c4bc-49ec-89c3-de4f4ff1585d}.pdf
Текст извлечен и сохранен в: pdf_files\{9c4620ae-c4bc-49ec-89c3-de4f4ff1585d}.txt
Скачан: {1de9e9b5-d88f-493c-8132-6992c24fcc6d}.pdf
Текст извлечен и сохранен в: pdf_files\{1de9e9b5-d88f-493c-8132-6992c24fcc6d}.txt
Скачан: {6d29aae1-ee4e-4827-9b2a-48d6ab13f932}.pdf
Текст извлечен и сохранен в: pdf_files\{6d29aae1-ee4e-4827-9b2a-48d6ab13f932}.txt
Скачан: {d730a129-50bc-4c3c-bdac-41ce16712f7c}.pdf
Текст извлечен и сохранен в: pdf_files\{d730a129-50bc-4c3c-bdac-41ce16712f7c}.txt
Скачан: {1b53f388-43b3-4485-b1b7-0af7eefa7daf}

c:\Users\Max\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:3406: DecompressionBombWarning: Image size (100155852 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


KeyboardInterrupt: 